In [13]:
from music21 import *
import glob
import pickle
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [96]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("Pokemon MIDIs/Pokemon - Littleroot Town.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                print(element)
                notes.append(str(element.pitch))
            if isinstance(element, chord.Chord):
                print(element)
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [97]:
chords = get_notes()

Parsing Pokemon MIDIs/Pokemon - Littleroot Town.mid
<music21.chord.Chord C5 E2>
<music21.note.Note F>
<music21.note.Note E>
<music21.note.Note C>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note G>
<music21.note.Note F>
<music21.note.Note C>
<music21.note.Note A>
<music21.chord.Chord G5 F2>
<music21.chord.Chord A5 C3>
<music21.note.Note G>
<music21.note.Note F>
<music21.note.Note C>
<music21.note.Note A>
<music21.note.Note A>
<music21.chord.Chord B-5 G2>
<music21.note.Note C>
<music21.chord.Chord C6 A2>
<music21.note.Note E>
<music21.note.Note C>
<music21.chord.Chord D6 A2>
<music21.note.Note E>
<music21.note.Note A>
<music21.note.Note C#>
<music21.note.Note A>
<music21.note.Note C>
<music21.note.Note A>
<music21.note.Note A>
<music21.chord.Chord C#6 G2>
<music21.note.Note G>
<music21.note.Note D>
<music21.note.Note D>
<music21.note.Note A>
<music21.note.Note E>
<music21.note.Note F>
<music21.note.Note A>
<music21.note.Note A>
<music21.note.Note D>
<music21.note.Note C#>


<music21.chord.Chord F5 E2>
<music21.note.Note G>
<music21.chord.Chord G5 C2>
<music21.chord.Chord C5 G2>
<music21.note.Note G>
<music21.chord.Chord C6 F3>
<music21.note.Note E>
<music21.chord.Chord B-5 G2>
<music21.note.Note G>
<music21.note.Note G>
<music21.note.Note F>
<music21.note.Note A>
<music21.note.Note F>
<music21.chord.Chord G5 F2>
<music21.note.Note F>
<music21.chord.Chord A5 C3>
<music21.note.Note G>
<music21.note.Note F>
<music21.note.Note A>
<music21.note.Note F>
<music21.chord.Chord B-5 G2>
<music21.note.Note A>
<music21.note.Note A>
<music21.note.Note D>
<music21.note.Note C>
<music21.note.Note A>
<music21.chord.Chord D6 A2>
<music21.chord.Chord A5 C#3>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note A>
<music21.note.Note E>
<music21.chord.Chord C#6 G2>
<music21.note.Note C#>
<music21.note.Note D>
<music21.note.Note D>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note A>
<music21.note.Note F>
<music21.note.Note C#>
<music21.note.Note A>
<mus

In [93]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
create_midi(chords, "chords")

In [55]:
import pygame
def play_music(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
    except pygame.error:
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)
# pick a midi music file you have ...
# (if not in working folder use full path)
music_file = "filename.mid"
freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)
# optional volume 0 to 1.0
pygame.mixer.music.set_volume(0.8)
try:
    play_music(music_file)
except KeyboardInterrupt:
    # if user hits Ctrl/C then exit
    # (works only in console mode)
    pygame.mixer.music.fadeout(1000)
    pygame.mixer.music.stop()
    raise SystemExit
play_music("chords.mid")

KeyboardInterrupt: 

In [ ]:
stop_m